In [1]:
# Render our plots inline
%matplotlib inline
%pylab inline  
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# General Plotting Parameters
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['font.size'] = 14.
mpl.rcParams['legend.fontsize'] = 12.
mpl.rcParams['axes.labelsize'] = 12.
mpl.rcParams['xtick.labelsize'] = 10.
mpl.rcParams['ytick.labelsize'] = 10.
mpl.rcParams['xtick.minor.pad'] = 4
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'

#Git says this is patched, but it doesn't work from Pip --upgrade 26-mar-2015
#mpl.rcParams['xtick.minor.visible'] = True  

# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
             (255, 187, 120), (44, 160, 44), (152, 223, 138),
              (148, 103, 189),
             (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127),
             (199, 199, 199), (188, 189, 34), (219, 219, 141),
             (23, 190, 207), (158, 218, 229),(214, 39, 40), (255, 152, 150)]  
    
# Scale the RGB values to the [0, 1] range,
# which is the format matplotlib accepts.  
for i in range(len(tableau20)): 
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

Populating the interactive namespace from numpy and matplotlib


In [2]:
import calc_enrich
reload(calc_enrich)
from calc_enrich import calc_del_U
from calc_enrich import machines_per_cascade
from calc_enrich import N_product_by_alpha
from calc_enrich import machines_per_enr_stage
from calc_enrich import product_per_enr_stage
from calc_enrich import machines_per_strip_stage
from calc_enrich import N_waste_by_alpha



from calc_enrich import Npc_from_Nstages
from calc_enrich import Nwc_from_Nstages
from calc_enrich import calc_feed_flows
from calc_enrich import find_N_stages
from calc_enrich import design_cascade

In [3]:
# centrifuge params 
eff = 1.0 # centrifuge efficiency

d = 0.15  # m 
Z = 0.5   # m
F_m = 15/1e6
T = 320.0# K
cut = 0.5
omega = 485/(d/2)

# cascade params
Nfc = 0.0071
Npc = 0.035
Nwc = 0.001
Fc_month = 739 #kg/month

# The feed enrichment to be used when using a cascade designed for LEU to produce HEU.
Nfc_mod = 0.20
Fc_month_mod = Fc_month #kg/month

#unit conversions
kg_sec2kg_month = 30.4*24*60*60
v_a = omega * (d/2.0)
Fc = Fc_month/kg_sec2kg_month
Fc_mod = Fc_month_mod/kg_sec2kg_month

# Comparison numbers (not needed for calculations)
del_U_th_yr = 1.1 #swu/yr
del_U_obs_yr = 0.71 #Swu/yr
del_U_th = del_U_th_yr/(365.25*24*60*60) #kgSWU/sec
del_U_obs = del_U_obs_yr/(365.25*24*60*60)

Pc_month = 77 #kg/month
Pc = Pc_month/kg_sec2kg_month

qty_len = 8
assay_len = 4

In [4]:
alpha, del_U, del_U_yr, dirac = calc_del_U(v_a, Z, d, F_m, T, cut, eff)
print "alpha", alpha
print "del_U", del_U
F_m

alpha 1.16321031431
del_U 7.03232816847e-08


1.5e-05

In [5]:
n_mach = machines_per_cascade(del_U, Npc, Nwc, Fc, Pc)
print "machines per cascade", n_mach

machines per cascade 25990.3926986


In [6]:
nenr, nstr = find_N_stages(alpha, Nfc, Npc, Nwc)
print "N enrich stage", nenr
print "N strip stage", nstr

mod_p_assay = Npc_from_Nstages(alpha, Nfc_mod, nenr)
mod_w_assay = Nwc_from_Nstages(alpha, Nfc_mod, nstr)
print "mod product assay", mod_p_assay
print "mod waste assay", mod_w_assay

N enrich stage 11
N strip stage 13
mod product assay 0.600852653247
mod waste assay 0.029084658345


In [7]:
product_assay = N_product_by_alpha(alpha, Nfc)
n_enr_mach= machines_per_enr_stage(alpha, del_U, Fc)
stage_product= product_per_enr_stage(alpha, Nfc, product_assay, Fc)

print "product assay stage", product_assay
print "n enrich mach per stage", n_enr_mach
print "stage product", stage_product

enrich_waste = Fc - stage_product
enr_waste_assay = (Fc*Nfc - stage_product*product_assay)/enrich_waste

n_strip_mach = machines_per_strip_stage(alpha, del_U, enrich_waste)
waste_assay = N_waste_by_alpha(alpha, enr_waste_assay)
print "n strip mach", n_strip_mach
print "waste assay", waste_assay

#waste_assay = waste_per_strip_stage(alpha, enr_waste_assay, enrich_waste)


product assay stage 0.00824923407502
n enrich mach per stage 53.2872354761
stage product 0.000140841318721
n strip mach 26.6127432942
waste assay 0.00511779135645


In [8]:
fa = 0.10
pa = 0.20
wa = 0.05

n_enr_st, n_str_st = find_N_stages(alpha, fa, pa, wa)
feed_flows = calc_feed_flows(n_enr_st, n_str_st, Fc, cut)
print "feed flows \n", feed_flows, "\n\n"

tot_stage_info, tot_machiens = design_cascade(cut, alpha, del_U,fa, feed_flows,
                   n_enr_st, n_str_st,
                   assay_len=4, qty_len=6, verbose=True, pretty=True)

tot_stages = n_enr_st+n_str_st


feed flows 
[ 0.00030693  0.00061387  0.0009208   0.00122774  0.00153467  0.00127889
  0.00102311  0.00076734  0.00051156  0.00025578] 


Stage   #Mach	 Feed    Product  Waste	 F_assay 	P_assay W_assay
0 	291 	4030.91    2015.45    2015.45    0.1 	0.1145 	0.0872
1 	243 	3359.09    1679.55    1679.55    0.1145 	0.1307 	0.1
2 	194 	2687.27    1343.64    1343.64    0.1307 	0.1488 	0.1145
3 	146 	2015.45    1007.73    1007.73    0.1488 	0.169 	0.1307
4 	97 	1343.64    671.82    671.82    0.169 	0.1913 	0.1488
5 	49 	671.82    335.91    335.91    0.1913 	0.2158 	0.169
-1 	233 	3224.73    1612.36    1612.36    0.0872 	0.1 	0.0759
-2 	175 	2418.55    1209.27    1209.27    0.0759 	0.0872 	0.0659
-3 	117 	1612.36    806.18    806.18    0.0659 	0.0759 	0.0572
-4 	59 	806.18    403.09    403.09    0.0572 	0.0659 	0.0496


In [9]:
## Design Cascade based on available number of centrifuges, determine max Fc
max_centrifuges = 1000
curr_feed = Fc
optimal_feed = Fc
step_size = 1.05  # MUST be a number Greater than 1. 
step = 1
#ideal_enrich_stage, ideal_strip_stage = find_N_stages(alpha, fa, pa, wa)
#print "n stages", ideal_enrich_stage, ideal_strip_stage
#feed_flows = calc_feed_flows(ideal_enrich_stage, ideal_strip_stage, curr_feed, cut)
all_stages, n_centrifuges = design_cascade(cut, alpha, del_U, fa, feed_flows, n_enr_st, n_str_st)

optimum_number = False
neg_inc = False
pos_inc = False

if (n_centrifuges < max_centrifuges):
    pos_inc = True
    step = step_size
elif (n_centrifuges > max_centrifuges):
    neg_inc = True
    step = 1.0/step_size
else:
    optimum_number = True
ntries=0
print "step size is", step
while (optimum_number == False) and (ntries < 100):
    ntries+=1
    last_feed = curr_feed
    curr_feed *= step
    feed_flows = calc_feed_flows(n_enr_st, n_str_st, curr_feed, cut)
    all_stages, n_centrifuges = design_cascade(cut, alpha, del_U, Nfc, feed_flows, n_enr_st, 
                                               n_str_st)
    print "curr try", ntries, "Tot mach", n_centrifuges, "# in last stage: ", all_stages[-1][1]
    if (all_stages[-1][1] < 1):
        print "not enough centrifuges to fill all stages"
        optimum_number = True
        optimal_feed = last_feed
    elif (neg_inc == True) and (n_centrifuges < max_centrifuges):
        optimum_number = True
        optimal_feed = curr_feed
        print "reduced flow to correct #", n_centrifuges
    elif (pos_inc == True) and (n_centrifuges > max_centrifuges):
        optimum_number = True
        optimal_feed = last_feed
        print "increased flow to correct #", n_centrifuges

final_feed_flows = calc_feed_flows(n_enr_st, n_str_st, optimal_feed, cut)
final_stages, final_n_mach = design_cascade(cut, alpha, del_U, Nfc, final_feed_flows, 
                                            n_enr_st, n_str_st, verbose=True, pretty=True)
print "Cascade can process ", optimal_feed*kg_sec2kg_month, " kg/month"
print "Final total machines", final_n_mach


step size is 0.952380952381
curr try 1 Tot mach 1528 # in last stage:  47
curr try 2 Tot mach 1453 # in last stage:  44
curr try 3 Tot mach 1386 # in last stage:  42
curr try 4 Tot mach 1320 # in last stage:  40
curr try 5 Tot mach 1256 # in last stage:  38
curr try 6 Tot mach 1198 # in last stage:  37
curr try 7 Tot mach 1141 # in last stage:  35
curr try 8 Tot mach 1087 # in last stage:  33
curr try 9 Tot mach 1035 # in last stage:  32
curr try 10 Tot mach 986 # in last stage:  30
reduced flow to correct # 986
Stage   #Mach	 Feed    Product  Waste	 F_assay 	P_assay W_assay
0 	179 	2474.63    1237.31    1237.31    0.0071 	0.0082 	0.0061
1 	149 	2062.19    1031.1    1031.1    0.0082 	0.0096 	0.0071
2 	119 	1649.75    824.88    824.88    0.0096 	0.0111 	0.0082
3 	90 	1237.31    618.66    618.66    0.0111 	0.0129 	0.0096
4 	60 	824.88    412.44    412.44    0.0129 	0.015 	0.0111
5 	30 	412.44    206.22    206.22    0.015 	0.0174 	0.0129
-1 	143 	1979.7    989.85    989.85    0.0061 	0.00

In [10]:
print "optimal feed ",optimal_feed


optimal feed  0.000172728547746
